# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key

# Configure gmaps
gmaps.configure(api_key=g_key)

# Google places api base url
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
file = "Output/weather_data.csv"
cities_df = pd.read_csv(file)
cities_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Albany,42.6001,-73.9662,70.00,18,87,4.00,US,1619290801
1,Revda,56.8010,59.9303,44.60,39,0,8.95,RU,1619290820
2,Cidreira,-30.1811,-50.2056,73.62,72,8,12.46,BR,1619290821
3,Punta Arenas,-53.1500,-70.9167,51.80,71,75,16.11,CL,1619290643
4,Östersund,63.1792,14.6357,33.01,51,61,12.66,SE,1619290821
...,...,...,...,...,...,...,...,...,...
572,Yuxia,34.0615,108.6291,53.60,87,40,8.95,CN,1619290978
573,Eureka,40.8021,-124.1637,53.01,84,100,7.00,US,1619290807
574,North Bend,43.4065,-124.2243,55.99,71,90,6.91,US,1619290842
575,Stabat,3.7610,98.4513,77.00,100,40,2.30,ID,1619290978


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
coords = cities_df[["Lat", "Lng"]]
figure_layout = {
    'width': '750px',
    'height': '600px',
    'border': '1px solid black',
    'padding': '1px',
}

In [10]:
fig = gmaps.figure(layout=figure_layout, zoom_level=1.5, center=(0,0), map_type="TERRAIN")

heatLayer = gmaps.heatmap_layer(coords, weights=cities_df["Humidity"], max_intensity=110, point_radius=1.4, dissipating=False)
fig.add_layer(heatLayer)

fig

Figure(layout=FigureLayout(border='1px solid black', height='600px', padding='1px', width='750px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
vacationSpots = cities_df.dropna()
vacationSpots = vacationSpots[vacationSpots.Humidity < 75]
vacationSpots = vacationSpots[vacationSpots.Humidity > 25]
vacationSpots = vacationSpots[vacationSpots["Max Temp"] > 70]
vacationSpots = vacationSpots[vacationSpots["Max Temp"] < 85]
vacationSpots = vacationSpots[vacationSpots.Cloudiness < 10]
vacationSpots = vacationSpots[vacationSpots["Wind Speed"] < 8]
vacationSpots

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
23,Ampanihy,-24.7000,44.7500,72.07,69,5,6.17,MG,1619290826
91,Umluj,25.0213,37.2685,83.07,50,0,5.55,SA,1619290845
192,Nūh,28.1167,77.0167,78.80,27,0,3.44,IN,1619290873
207,Makakilo City,21.3469,-158.0858,78.80,69,1,6.91,US,1619290877
382,Erzin,36.9559,36.2000,71.60,46,0,4.61,TR,1619290926
412,Vicuña,-30.0319,-70.7081,73.00,30,0,3.00,CL,1619290933


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
hotel_df = vacationSpots#[["City", "Lat", "Lng", "Country"]]
hotelNames = []

for lat, lng in zip(hotel_df.Lat, hotel_df.Lng):
    target_coords = f"{lat}, {lng}"
    params = {
        "location": target_coords,
        "radius": 5000,
        "type": "hotel",
        "key": g_key
    }
    response = requests.get(base_url, params=params)
    # convert response to json
    places_data = response.json()
    hotelNames.append(places_data['results'][0]['name'])

In [7]:
hotel_df["Hotel Name"] = hotelNames
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
23,Ampanihy,-24.7000,44.7500,72.07,69,5,6.17,MG,1619290826,Ampanihy
91,Umluj,25.0213,37.2685,83.07,50,0,5.55,SA,1619290845,Umluj
192,Nūh,28.1167,77.0167,78.80,27,0,3.44,IN,1619290873,Nuh
207,Makakilo City,21.3469,-158.0858,78.80,69,1,6.91,US,1619290877,Kapolei
382,Erzin,36.9559,36.2000,71.60,46,0,4.61,TR,1619290926,Erzin
412,Vicuña,-30.0319,-70.7081,73.00,30,0,3.00,CL,1619290933,Vicuña


In [8]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [9]:
fig = gmaps.figure(layout=figure_layout, zoom_level=1.5, center=(0,0), map_type="TERRAIN")
heatLayer = gmaps.heatmap_layer(coords, weights=cities_df["Humidity"], max_intensity=110, point_radius=1.4, dissipating=False)
fig.add_layer(heatLayer)

#Create Marker Layer
markerLayer = gmaps.marker_layer(locations, hotel_info)
# Add marker layer ontop of heat map
fig.add_layer(markerLayer)

# Display figure
fig

Figure(layout=FigureLayout(border='1px solid black', height='600px', padding='1px', width='750px'))